In [2]:
BUCKET_NAME="feature_store_rhun"
BIGQUERY_DATASET_NAME="store_attrs"

In [12]:
!gsutil mb gs://$BUCKET_NAME

Creating gs://feature_store_rhun/...
ServiceException: 409 A Cloud Storage bucket named 'feature_store_rhun' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [13]:
!bq mk $BIGQUERY_DATASET_NAME

BigQuery error in mk operation: Dataset 'fast-envoy-329104:store_attrs' already
exists.


In [3]:
! feast init --minimal --template gcp feast_demo 
%cd feast_demo
! ls


Creating a new Feast repository in /home/jupyter/feast_demo.

/home/jupyter/feast_demo
feature_store.yaml


In [4]:
feature_store = \
f"""project: feast_demo
registry: gs://{BUCKET_NAME}/registry.db
provider: gcp"""

with open('feature_store.yaml', "w") as feature_store_file:
    feature_store_file.write(feature_store)

# Print our feature_store.yaml
! cat feature_store.yaml

project: feast_demo
registry: gs://feature_store_rhun/registry.db
provider: gcp

In [22]:
# !pip install feast==0.10.5 pandas==1.3.5

In [5]:
import pandas as pd
df = pd.read_parquet('../features.parquet')

In [6]:
list1 = []
for x in range(0,100):
    list1.append(x)
df['created_on'] = pd.Series(pd.to_datetime(list1, unit='D', origin=pd.Timestamp('2022-01-01')))

In [7]:
df.to_gbq('fast-envoy-329104.demo_data.store_attrs', 
          chunksize=None, # I have tried with several chunk sizes, it runs faster when it's one big chunk (at least for me)
          if_exists='replace'
         )

100%|██████████| 1/1 [00:00<00:00, 8774.69it/s]


In [8]:
%%bash
FEATURES_PY=features.py
if test -f $FEATURES_PY; then
    rm $FEATURES_PY
fi
touch $FEATURES_PY
echo -e 'from feast import Entity, Feature, FeatureView, ValueType, BigQuerySource
from datetime import timedelta

# Define an entity for the project
store_attrs = Entity(
    name="store_sk_id",
    join_key="store_sk_id",
    value_type=ValueType.INT64,
)

# Read data
store_attrs_source = BigQuerySource(
    query="SELECT * `fast-envoy-329104.demo_data.store_attrs`",
    event_timestamp_column="created_on"
)

# Define a Feature View for each project
# Can be used for fetching historical data and online serving
store_attrs_view = FeatureView(
    name="store_details",
    entities=["store_sk_id"],
    ttl=timedelta(weeks=52),
    features=[
        Feature(name="current_ind", dtype=ValueType.STRING),
        Feature(name="store_nbr", dtype=ValueType.INT64),
        Feature(name="store_name", dtype=ValueType.STRING),
        Feature(name="street_addr_line1", dtype=ValueType.STRING),
        Feature(name="city_name", dtype=ValueType.STRING),
        Feature(name="state_prov_code", dtype=ValueType.STRING),
        Feature(name="postal_code", dtype=ValueType.STRING),
        Feature(name="subdiv_nbr", dtype=ValueType.STRING),
        Feature(name="subdiv_name", dtype=ValueType.STRING),
        Feature(name="region_name", dtype=ValueType.STRING),
        Feature(name="region_nbr", dtype=ValueType.INT64),
        Feature(name="market_area_nbr", dtype=ValueType.INT64),
        Feature(name="market_area_name", dtype=ValueType.STRING),
        Feature(name="primary_trait_name", dtype=ValueType.STRING),
        Feature(name="county_name", dtype=ValueType.STRING),
        Feature(name="phone_nbr", dtype=ValueType.STRING),
        Feature(name="store_size_qty", dtype=ValueType.INT64),
        Feature(name="store_size_uom_code", dtype=ValueType.STRING),
        Feature(name="store_size_uom_desc", dtype=ValueType.STRING),
        Feature(name="open_sunday_ind", dtype=ValueType.STRING),
        Feature(name="open_status_code", dtype=ValueType.STRING),
        Feature(name="open_status_desc", dtype=ValueType.STRING),
        Feature(name="expansion_size_qty", dtype=ValueType.INT64),
        Feature(name="real_store_nbr", dtype=ValueType.INT64),
        Feature(name="temp_store_nbr", dtype=ValueType.INT64),
        Feature(name="temp_store_ind", dtype=ValueType.STRING),
        Feature(name="store_type_code", dtype=ValueType.STRING),
        Feature(name="store_type_desc", dtype=ValueType.STRING),
        Feature(name="size_class_code", dtype=ValueType.INT64),
        Feature(name="sales_class_code", dtype=ValueType.INT64),
        Feature(name="store_comp_code", dtype=ValueType.STRING),
        Feature(name="store_comp_desc", dtype=ValueType.STRING),
        Feature(name="store_comp_ind", dtype=ValueType.STRING),
        Feature(name="financial_rpt_code", dtype=ValueType.STRING),
        Feature(name="financial_rpt_desc", dtype=ValueType.STRING),
        Feature(name="delivery_type_code", dtype=ValueType.STRING),
        Feature(name="delivery_type_desc", dtype=ValueType.STRING),
        Feature(name="prototype_nbr", dtype=ValueType.STRING),
        Feature(name="time_zone_code", dtype=ValueType.STRING),
        Feature(name="latitude_dgr", dtype=ValueType.FLOAT),
        Feature(name="longitude_dgr", dtype=ValueType.FLOAT),
        Feature(name="apparel_zone_nbr", dtype=ValueType.INT64),
        Feature(name="mdse_major_zone_nbr", dtype=ValueType.INT64),
        Feature(name="mdse_sub_zone_nbr", dtype=ValueType.INT64),
        Feature(name="banner_code", dtype=ValueType.STRING),
        Feature(name="banner_desc", dtype=ValueType.STRING),
        Feature(name="store_mgr_name", dtype=ValueType.STRING),
        Feature(name="buo_area_nbr", dtype=ValueType.INT64),
        Feature(name="base_div_nbr", dtype=ValueType.INT64),
        Feature(name="buo_area_name", dtype=ValueType.STRING),
        Feature(name="subregion_nbr", dtype=ValueType.STRING),
        Feature(name="subregion_name", dtype=ValueType.STRING),
        Feature(name="dec_store_nbr", dtype=ValueType.STRING),
        Feature(name="buo_area_mgr_name", dtype=ValueType.STRING),
        Feature(name="subdiv_mgr_name", dtype=ValueType.STRING),
        Feature(name="region_mgr_name", dtype=ValueType.STRING),
        Feature(name="subregion_mgr_name", dtype=ValueType.STRING),
        Feature(name="market_area_mgr_name", dtype=ValueType.STRING),
        Feature(name="geo_point_nbr", dtype=ValueType.STRING)
    ],
    input=store_attrs_source,
    tags={"team": "my_team"},
)
' >> $FEATURES_PY

In [10]:
!feast apply

In [14]:
import pandas as pd
from feast import FeatureStore

In [15]:
# Identify entities
project_ids = df.store_sk_id[0:3].to_list()
now = datetime.now()
timestamps = [datetime(now.year, now.month, now.day)]*len(project_ids)
entity_df = pd.DataFrame.from_dict({"store_sk_id": project_ids, "event_timestamp": timestamps})
entity_df.head()

,store_sk_id,event_timestamp
0,752,2022-07-27
1,127,2022-07-27
2,801,2022-07-27


In [16]:
# Get historical features
store = FeatureStore(repo_path="features")
training_df = store.get_historical_features(
    entity_df=entity_df,
    feature_refs=["store_details:subregion_mgr_name", "store_details:latitude_dgr"],
).to_df()
training_df.head()

,event_timestamp,store_sk_id,store_details__subregion_mgr_name,store_details__latitude_dgr
0,2022-07-27 00:00:00+00:00,752,lnyirfnhrw,0.274
1,2022-07-27 00:00:00+00:00,127,oubcruropm,0.123
2,2022-07-27 00:00:00+00:00,801,buxoxljghc,0.612
